### K-means1

In [ ]:
def create_kmeans_mask(image):
    # Reshape the image to a 2D array of pixels and 3 color values (RGB)
    pixel_vals = image.reshape((-1, 3))

    # Convert to float type
    pixel_vals = np.float32(pixel_vals)

    # Define criteria for algorithm termination
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.85)

    # Perform k-means clustering
    k = 5
    _, labels, centers = cv2.kmeans(pixel_vals, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert data into 8-bit values
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]

    # Reshape data into the original image dimensions
    segmented_image = segmented_data.reshape(image.shape)

    # Convert to grayscale
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)

    # Apply thresholding to create binary image
    _, binary_image = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    return binary_image

In [ ]:
# Example usage
kmeans_images = batch_process_images(lab_images, create_kmeans_mask)

Processing daun_menguning: 100%|██████████| 45/45 [00:04<00:00,  9.00it/s]


### K-means2

In [ ]:
def create_kmeans_mask_sklearn(image):
    # Reshape the image to a 2D array of pixels and 3 color values (RGB)
    pixel_vals = image.reshape((-1, 3))

    # Convert to float type
    pixel_vals = np.float32(pixel_vals)

    # Perform k-means clustering
    k = 10
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pixel_vals)

    # Get the labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_

    # Convert data into 8-bit values
    centers = np.uint8(centers)
    segmented_data = centers[labels.flatten()]

    # Reshape data into the original image dimensions
    segmented_image = segmented_data.reshape(image.shape)

    # Convert to grayscale
    gray = cv2.cvtColor(segmented_image, cv2.COLOR_RGB2GRAY)

    # Apply thresholding to create binary image
    _, binary_image = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    return binary_image

In [ ]:
# Example usage
kmeans2_images = batch_process_images(lab_images, create_kmeans_mask_sklearn)

Processing daun_menguning: 100%|██████████| 45/45 [00:25<00:00,  1.76it/s]


### K-means5

In [ ]:
def create_main_leaf_mask(image_lab):
    # Reshape the image to a 2D array of pixels and 3 color values (LAB)
    pixel_vals = image_lab.reshape((-1, 3))

    # Perform k-means clustering
    k = 3  # Increased to 3 clusters for better separation
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(pixel_vals)

    # Get the labels and cluster centers
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_

    # Convert centers to uint8
    centers = np.uint8(centers)

    # Create segmented image
    segmented_image = centers[labels.reshape(image_lab.shape[:2])]

    # Convert back to BGR color space for visualization (if needed)
    segmented_bgr = cv2.cvtColor(segmented_image, cv2.COLOR_LAB2BGR)

    # Create masks for each cluster
    masks = [np.all(segmented_image == center, axis=2) for center in centers]

    # Find the mask with the largest connected component
    largest_mask = None
    largest_area = 0

    for mask in masks:
        # Convert bool mask to uint8
        mask_uint8 = mask.astype(np.uint8) * 255
        
        # Find connected components
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask_uint8, connectivity=8)
        
        # Skip the background label (0)
        for label in range(1, num_labels):
            area = stats[label, cv2.CC_STAT_AREA]
            if area > largest_area:
                largest_area = area
                largest_mask = (labels == label).astype(np.uint8) * 255

    # Apply morphological operations to clean up the mask
    kernel = np.ones((5,5), np.uint8)
    largest_mask = cv2.morphologyEx(largest_mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    largest_mask = cv2.morphologyEx(largest_mask, cv2.MORPH_OPEN, kernel, iterations=2)

    return largest_mask


In [ ]:
# Example usage
kmeans5_images = batch_process_images(lab_images, create_main_leaf_mask)

Processing daun_menguning: 100%|██████████| 45/45 [00:06<00:00,  6.60it/s]
